# Day 1 - Prompting Basics

In this notebook we cover the basic ideas of prompting including zero-shot prompting, few-shot prompting, prompt chaining, system prompts and memory. 

In [1]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [2]:
from tools import llm_call

## Context Length/Size

Context length refers to the maximum number of tokens we can feed as input to a model. Roughly speaking 1 token = 3/4 of a word. We list the context sizes for the models we use below:

* `gpt-4`: 8192 tokens
* `gpt-3.5-turbo`: 4097 tokens
* `text-bison`: 8192 tokens
* `chat-bison`: 8192 tokens
* `llama-2`: 4096 tokens

Note that in general the entirety of the input you send to the model counts towards this context size. This includes the prompt, the system prompt (explained below) and the chat history.

You can find more information about OpenAI's models [here](https://platform.openai.com/docs/models/overview), Google's models [here](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models), Anthropic's models [here](https://docs.anthropic.com/claude/reference/selecting-a-model) and Meta's models [here](https://ai.meta.com/llama/).

## Prompting

Prompting is something many people are familiar with already. You send in some plain text to the model and you get back an output.

In [3]:
prompt = "Who played Saruman in the Lord of the Rings trilogy?"

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
Who played Saruman in the Lord of the Rings trilogy?
---------RESPONSE---------
Christopher Lee played Saruman in the Lord of the Rings trilogy.


## Zero-Shot Prompting

Zero-shot prompting refers to a method of prompting LLMs to perform a task by directly asking it to do so, with no examples in the prompt. To demonstrate this, let's consider a more specific use case - say classification.

In [4]:
prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative".

Review: I loved the new Spider-Man movie!! The animation was really fluid
Sentiment: 
"""

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
Classify the sentiment of the following review as "positive", "neutral" or "negative".

Review: I loved the new Spider-Man movie!! The animation was really fluid
Sentiment: 

---------RESPONSE---------
Positive


## Few-Shot Prompting

Few-shot prompting takes zero-shot prompting and adds a couple of examples for the model to learn from. This can help guide the model in terms of getting it to respond in a certain way, helping it in better answering prompts or just showing it an example on how to do something it may have not seen before.

As a trivial example, the response above returns "Positive" instead of "positive" even though we explicitly ask for the latter. With few-shot prompting, we can fix that!

In [8]:
prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative".

Review: I didn't think the new Spider-Man movie was that great. It was a decent watch, had nice animation.
Sentiment: neutral

Review: The new Spider-Man movie was boring. I just can't watch animated movies..
Sentiment: negative

Review: I loved the new Spider-Man movie!! The animation was really fluid
Sentiment: 
"""

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
Classify the sentiment of the following review as "positive", "neutral" or "negative".

Review: I didn't think the new Spider-Man movie was that great. It was a decent watch, had nice animation.
Sentiment: neutral

Review: The new Spider-Man movie was boring. I just can't watch animated movies..
Sentiment: negative

Review: I loved the new Spider-Man movie!! The animation was really fluid
Sentiment: 

---------RESPONSE---------
positive


## Prompt Chaining

Sometimes one prompt is not enough. What if we had a scenario where we wanted to use the output of one prompt as an input to another prompt? We can chain them together! This concept is the basis for the idea of "chains" popularized by LangChain (which we'll cover tomorrow). This is also used in techniques like least-to-most prompting where we break down tasks into smaller subtasks and use prompts to solve them one by one. 

Consider the following example where we first task the model to extract information from a given document and then query the model based on the extracted information.

In [9]:
paragraph = """Pizza (English: /ˈpiːtsə/ PEET-sə, Italian: [ˈpittsa], Neapolitan: [ˈpittsə]) is a dish of Italian origin consisting of a usually round, flat base of leavened wheat-based dough topped with tomatoes, cheese, and often various other ingredients (such as various types of sausage, anchovies, mushrooms, onions, olives, vegetables, meat, ham, etc.), which is then baked at a high temperature, traditionally in a wood-fired oven.[1]

The term pizza was first recorded in the 10th century in a Latin manuscript from the Southern Italian town of Gaeta in Lazio, on the border with Campania.[2] Raffaele Esposito is often credited for creating modern pizza in Naples.[3][4][5][6] In 2009, Neapolitan pizza was registered with the European Union as a Traditional Speciality Guaranteed dish. In 2017, the art of making Neapolitan pizza was added to UNESCO's list of intangible cultural heritage.[7]

Pizza and its variants are among the most popular foods in the world. Pizza is sold at a variety of restaurants, including pizzerias (pizza specialty restaurants), Mediterranean restaurants, via delivery, and as street food.[8] In Italy, pizza served in a restaurant is presented unsliced, and is eaten with the use of a knife and fork.[9][10] In casual settings, however, it is cut into wedges to be eaten while held in the hand. Pizza is also sold in grocery stores in a variety of forms, including frozen or as kits for self-assembly. They are then cooked using a home oven.

In 2017, the world pizza market was US$128 billion, and in the US it was $44 billion spread over 76,000 pizzerias.[11] Overall, 13% of the U.S. population aged two years and over consumed pizza on any given day.[12]"""

In [10]:
prompt = f"List out all the ingredients used to create the dish in the following paragraph. Do not include ingredients that are not in the paragraph.\nParagraph: {paragraph}"

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
output = llm_call(model="gpt-4", prompt=prompt)
print(output)

----------PROMPT----------
List out all the ingredients used to create the dish in the following paragraph. Do not include ingredients that are not in the paragraph.
Paragraph: Pizza (English: /ˈpiːtsə/ PEET-sə, Italian: [ˈpittsa], Neapolitan: [ˈpittsə]) is a dish of Italian origin consisting of a usually round, flat base of leavened wheat-based dough topped with tomatoes, cheese, and often various other ingredients (such as various types of sausage, anchovies, mushrooms, onions, olives, vegetables, meat, ham, etc.), which is then baked at a high temperature, traditionally in a wood-fired oven.[1]

The term pizza was first recorded in the 10th century in a Latin manuscript from the Southern Italian town of Gaeta in Lazio, on the border with Campania.[2] Raffaele Esposito is often credited for creating modern pizza in Naples.[3][4][5][6] In 2009, Neapolitan pizza was registered with the European Union as a Traditional Speciality Guaranteed dish. In 2017, the art of making Neapolitan piz

We can now chain the result of the previous prompt into our question.

In [12]:
prompt = f"From the following list of ingredients, classify them into vegetarian and non-vegetarian options. \nIngredients: \n{output}"

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
From the following list of ingredients, classify them into vegetarian and non-vegetarian options. 
Ingredients: 
- Wheat-based dough
- Tomatoes
- Cheese
- Sausage
- Anchovies
- Mushrooms
- Onions
- Olives
- Vegetables
- Meat
- Ham
---------RESPONSE---------
Vegetarian:
- Wheat-based dough
- Tomatoes
- Cheese
- Mushrooms
- Onions
- Olives
- Vegetables

Non-Vegetarian:
- Sausage
- Anchovies
- Meat
- Ham


While the above is a pretty simple example, we can imagine other cases where this would be useful. For example, let's say we have a 20 page document we want to summarize. It might not fit into the model entirely. Instead we could summarize each page of the document and then concatenate all the summaries to get a short version of the document. We can then summarize this new document to get a final summary.

## System Prompts

A system prompt is a prompt we can use to set the "behavior" of the LLM for the duration of the conversation. Typical use cases include giving the model a set of instructions to follow during all future interactions in the chat session. For instance, we could give the LLM a list of instructions for a classification task such as the potential classes, identifiers for each class, text descriptions or even examples. Then, as a prompt we would only need to give the model some input data to classify.

In general, the system prompt only exists in Chat models which are designed for conversations. Instruction fine-tuned models usually do not have conversations, so no system prompts are used there.

In the following example we consider the same prompt twice - once without a system prompt and once with it.

Note: Claude does not pay a lot of attention to the system prompt, as described [here](https://docs.anthropic.com/claude/docs/constructing-a-prompt#system-prompt-optional).

In [16]:
prompt = f"Who directed the Avatar film series?"

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
Who directed the Avatar film series?
---------RESPONSE---------
James Cameron directed the Avatar film series.


In [15]:
system_prompt = """You are a helpful chat assistant. Always follow these rules:
1. Speak in the style of a voiceover for an action movie.
2. Always end sentences with a joke. 
3. Use the words "If you please" in every response."""

prompt = f"Who directed the Avatar film series?"

print("------SYSTEM PROMPT-------")
print(system_prompt)
print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt, system_prompt=system_prompt))

------SYSTEM PROMPT-------
You are a helpful chat assistant. Always follow these rules:
1. Speak in the style of a voiceover for an action movie.
2. Always end sentences with a joke. 
3. Use the words "If you please" in every response.
----------PROMPT----------
Who directed the Avatar film series?
---------RESPONSE---------
In the swirling mists of Hollywood, one man stood tall, his vision piercing the veil of cinematic cliches. With a determined heart and a firm hand, he embarked on the journey, ruling with an iron gaze. That man, ladies and gentlemen, is none other than James Cameron, the mastermind behind the Avatar Film Series! He's been sinking the competition since Titanic. Now, that's a deep joke, if you please!


## Memory

In a conversational system, we would expect the model to be able to remember previous parts of the conversation and reference them during new responses. This requires the LLM to hold this information somewhere in memory. 

The simplest way to do this is to literally just store the entire conversation in a list and pass it along with each model call. 

Note: Only Chat models use this concept. To replicate the same with an instruction fine-tuned model, just add everything into the prompt.

In [10]:
system_prompt = "You are a helpful chat assistant."
prompt = "Who played Saruman in the Lord of the Rings trilogy?"

print("------SYSTEM PROMPT-------")
print(system_prompt)
print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
result, chat_history = llm_call(model="gpt-4", prompt=prompt, system_prompt=system_prompt, return_chat_history=True)
print(result)

------SYSTEM PROMPT-------
You are a helpful chat assistant.
----------PROMPT----------
Who played Saruman in the Lord of the Rings trilogy?
---------RESPONSE---------
Saruman in the Lord of the Rings trilogy was played by Sir Christopher Lee.


Now let's continue the conversation. First, let's look at what the chat history looks like:

In [11]:
chat_history

[{'role': 'system', 'content': 'You are a helpful chat assistant.'},
 {'role': 'user',
  'content': 'Who played Saruman in the Lord of the Rings trilogy?'},
 {'role': 'assistant',
  'content': 'Saruman in the Lord of the Rings trilogy was played by Sir Christopher Lee.'}]

In [12]:
system_prompt = "You are a helpful chat assistant."
prompt = "What are some facts about him?"

print("------SYSTEM PROMPT-------")
print(system_prompt)
print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
result, chat_history = llm_call(model="gpt-4", prompt=prompt, system_prompt=system_prompt, return_chat_history=True, chat_history=chat_history)
print(result)

------SYSTEM PROMPT-------
You are a helpful chat assistant.
----------PROMPT----------
What are some facts about him?
---------RESPONSE---------
Sir Christopher Lee was a British actor with a rich and varied career in film, television, and music, spanning over seven decades. Here are some facts about him:

1. He was born on May 27, 1922, in Belgravia, London, England, and died on June 7, 2015.

2. Lee is best known for his roles in horror films produced by Hammer Studios from the late 1950s. He played iconic roles like Count Dracula and Frankenstein's monster.

3. Apart from working in horror movies, he's also famous for his roles in The Lord of the Rings and The Hobbit trilogies as Saruman, and in the Star Wars prequel trilogy as Count Dooku.

4. He was knighted for services to drama and charity in 2009 and received the BAFTA Fellowship in 2011.

5. Lee was a step-cousin of Ian Fleming, author of the James Bond spy novels. He even played a Bond villain, Francisco Scaramanga, in "The 

Here, we see that the model uses the context from the previous conversation to remember that "him" refers to Sir Christopher Lee and generates some facts about him. Note: These facts may or may not be true and should be taken with a grain of salt.

### Taking memory further

In upcoming sessions, we'll take this idea of memory further with the introduction of vector databases and Retrieval Augmented Generation (RAG). We use these systems in scenarios where we have a lot of information to refer to and need a way of finding the most relevant information for the conversation at hand.